In [0]:
%pip install openpyxl
%pip install msoffcrypto-tool

In [0]:
import os
import re
import pandas as pd
from datetime import datetime
from pyspark.sql import functions as F

In [0]:
# List all files in the holdings directory and filter for Excel files
files = [f for f in dbutils.fs.ls("/Volumes/personal_finance/default/files/angel_one/holdings") if not f.isDir() and (f.name.endswith('.xlsx') or f.name.endswith('.xls'))]

# Extract client id and formatted datetime from each file
data = []
for f in files:
    match = re.search(r'Your_Holding_Details_([A-Z0-9]+)\.xlsx', f.name)
    client_id = match.group(1) if match else None
    dt = datetime.fromtimestamp(f.modificationTime / 1000).strftime('%Y-%m-%d %H:%M:%S')
    path = f.path
    if path.startswith("dbfs:"):
        path = path[5:]
    data.append({"name": f.name, "path": path, "modificationTime": dt, "client_id": client_id, "modTimeRaw": f.modificationTime})

file_info_df = pd.DataFrame(data)

# Keep only the most recently modified file for each client_id
file_info_df = file_info_df.sort_values("modTimeRaw", ascending=False).drop_duplicates(subset=["client_id"], keep="first").drop(columns=["modTimeRaw"])

display(file_info_df)

In [0]:
import pandas as pd
import msoffcrypto
import io
# Create a new DataFrame for client_id to password mapping
client_passwords_df = spark.sql("select * from personal_finance.bronze.angel_one_client_passwords").toPandas()
client_passwords = dict(zip(client_passwords_df['client_id'], client_passwords_df['password']))

dfs = []
for _, f in file_info_df.iterrows():
    client_id = f.get("client_id")
    password = client_passwords.get(client_id)

    with open(f["path"], "rb") as file:
        office_file = msoffcrypto.OfficeFile(file)
        office_file.load_key(password=password)
        decrypted = io.BytesIO()
        office_file.decrypt(decrypted)
        decrypted.seek(0)
        df = pd.read_excel(
            decrypted,
            engine="openpyxl", sheet_name="Equity", skiprows=14
        )
        dfs.append(df)

if dfs:
    merged_df = pd.concat(dfs, ignore_index=True)
    display(merged_df)

In [0]:
for i, df in enumerate(dfs):
    if "Client ID" in df.columns:
        dfs[i] = df[df["Client ID"] != "Total"]

if dfs:
    merged_df = pd.concat(dfs, ignore_index=True)
    display(merged_df)

In [0]:
spark_df = spark.createDataFrame(merged_df)
spark_df = spark_df.toDF(*[c.replace(" ", "_").replace("(","").replace(")","") for c in spark_df.columns])
spark_df.write.mode("overwrite").saveAsTable("personal_finance.bronze.angel_one_holding_statement")